In [1]:
## 导入相关库

准备构建神经网络模型所需的工具和环境

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle


## 设置超参数

In [3]:
# 图像参数
img_height = 32
img_width = 32
img_channels = 3

# GAN 参数
latent_dim = 100
generator_input_shape = (latent_dim,)
discriminator_input_shape = (img_height, img_width, img_channels)

# 训练参数
batch_size = 64
epochs = 500
learning_rate = 0.0002
beta_1 = 0.5


训练生成对抗网络（GAN）的关键参数。下面是各个参数的解释：
img_height: 图像的高度。在这个示例中，图像的高度被设置为32个像素。
img_width: 图像的宽度。在这个示例中，图像的宽度也被设置为32个像素。
img_channels: 图像的通道数。在这个示例中，图像的通道数被设置为3，代表RGB彩色图像。
latent_dim: 潜在空间的维度。潜在空间是生成器模型的输入，它是一个随机向量。在这个示例中，潜在空间的维度被设置为100。
generator_input_shape: 生成器模型的输入形状。在这个示例中，生成器的输入形状是一个大小为100的一维向量。
discriminator_input_shape: 判别器模型的输入形状。在这个示例中，判别器的输入形状是一个大小为(32, 32, 3)的三维张量，代表着32x32像素的RGB彩色图像。
batch_size: 每次训练时用于更新模型的样本数量。在这个示例中，每次训练使用64个样本进行更新。
epochs: 训练的轮数。在这个示例中，训练将进行500轮。
learning_rate: 学习率。学习率决定了模型参数在每次迭代中的更新幅度。在这个示例中，学习率被设置为0.0002。
beta_1: Adam优化器中的beta_1参数。Adam优化器是一种常用的优化算法，beta_1参数控制了梯度的一阶矩估计的衰减率。在这个示例中，beta_1被设置为0.5。

## 定义生成器和判别器

In [4]:
#生成器模型
def build_generator(input_shape):
    model = tf.keras.Sequential([
        layers.Dense(128 * 8 * 8, use_bias=False, input_shape=input_shape),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Reshape((8, 8, 128)),

        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model
#判别器模型
def build_discriminator(input_shape):
    model = tf.keras.Sequential([
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=input_shape),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Flatten(),
        layers.Dense(1)
    ])
    return model


layers.Dense：全连接层，将输入数据展平为12888维度。
layers.BatchNormalization：批归一化层，用于加速训练过程。
layers.LeakyReLU：LeakyReLU激活函数，通过引入负数部分来增强模型的表达能力。
layers.Reshape：重塑层，将数据重塑为8x8x128的形状。
layers.Conv2DTranspose：反卷积层，用于将数据从低维度映射到高维度。
layers.BatchNormalization和layers.LeakyReLU：再次使用批归一化层和LeakyReLU激活函数。
layers.Conv2DTranspose：再次使用反卷积层，将数据从低维度映射到3维度（RGB图像通道）。
activation='tanh'：最后一层使用tanh激活函数，将输出值限定在[-1,1]范围内。

## 初始化生成器和判别器，并定义损失函数和优化器

In [5]:
#初始化生成器和判别器
generator = build_generator(generator_input_shape)
discriminator = build_discriminator(discriminator_input_shape)

#定义损失函数
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

#定义优化器
generator_optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=beta_1)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=beta_1)


D:\Anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
D:\Anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


初始化生成器和判别器：调用了前面定义的build_generator和build_discriminator函数，分别创建了生成器（generator）和判别器（discriminator）模型。

定义损失函数：使用tf.keras.losses.BinaryCrossentropy函数创建了一个二分类交叉熵损失函数cross_entropy。
对于判别器，真实数据的损失是判别器输出与全1向量之间的交叉熵，虚假数据的损失是判别器输出与全0向量之间的交叉熵。
生成器的损失是判别器对虚假数据的输出与全1向量之间的交叉熵。

定义优化器：使用tf.keras.optimizers.Adam函数创建了两个Adam优化器，分别用于更新生成器和判别器的参数。
learning_rate参数控制学习率的大小，beta_1参数是Adam优化器中的指数衰减率，用于控制一阶矩估计的权重。

## 加载cifa-10数据集

In [6]:
def load_cifar10_data():
    data = []
    labels = []
    for i in range(1, 6):
        filename = f'C:/Users/Starfish/Desktop/单佳震/cifar-10-batches-py/data_batch_{i}'
        with open(filename, 'rb') as f:
            batch = pickle.load(f, encoding='bytes')
            data.append(batch[b'data'])
            labels += batch[b'labels']
    data = np.concatenate(data)
    data = data.reshape((-1, img_channels, img_height, img_width)).transpose(0, 2, 3, 1) / 255.0
    labels = np.array(labels)
    return data, labels

train_images, train_labels = load_cifar10_data()


## 定义训练步骤

In [7]:
@tf.function
def train_step(images):
    noise = tf.random.normal([batch_size, latent_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


## 训练模型

In [ ]:
from tqdm import tqdm
from time import sleep

for i in tqdm(range(100)):
    sleep(0.05)

def train(dataset, epochs):
    num_examples_to_generate = 32

    generator_losses = []
    discriminator_losses = []
    epoch_numbers = []

    for epoch in range(epochs):
        for i in range(0, len(dataset), batch_size):
            images = dataset[i:i+batch_size]
            train_step(images)

        if (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch+1}/{epochs}")

            noise = tf.random.normal([num_examples_to_generate, latent_dim])
            generated_images = generator(noise, training=False)

            plt.figure(figsize=(12, 6))
            for j in range(num_examples_to_generate):
                plt.subplot(4, 8, j+1)
                plt.imshow((generated_images[j, :, :, :] + 1) / 2)
                plt.axis('off')

            plt.show()

            noise = tf.random.normal([batch_size, latent_dim])
            generated_images = generator(noise, training=False)

            real_output = discriminator(images, training=False)
            fake_output = discriminator(generated_images, training=False)

            gen_loss = generator_loss(fake_output)
            disc_loss = discriminator_loss(real_output, fake_output)
            generator_losses.append(gen_loss)
            discriminator_losses.append(disc_loss)
            epoch_numbers.append(epoch + 1)

            print(f"Generator Loss: {gen_loss}")
            print(f"Discriminator Loss: {disc_loss}")

            
    return epoch_numbers, generator_losses, discriminator_losses


# 调用训练函数
epoch_numbers, generator_losses, discriminator_losses = train(train_images, epochs)


100%|████████████████████| 100/100 [00:05<00:00, 19.48it/s]


## 损失函数

In [ ]:
# 绘制损失曲线
plt.plot(epoch_numbers, generator_losses, label="Generator Loss")
plt.plot(epoch_numbers, discriminator_losses, label="Discriminator Loss")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Generator and Discriminator Losses-单佳震')
plt.legend()
plt.show()

## 模型评估

In [ ]:
def evaluate_gan(generator, discriminator, test_data):
    # 生成图像质量评估
    noise = tf.random.normal([10, latent_dim])
    generated_images = generator(noise, training=False)

    plt.figure(figsize=(12, 4))
    for i in range(10):
        plt.subplot(2, 10, i + 1)
        plt.imshow((generated_images[i] + 1) / 2)
        plt.title('Generated')
        plt.axis('off')
        
        plt.subplot(2, 10, i + 11)
        plt.imshow((test_data[i] + 1) / 2)
        plt.title('Real')
        plt.axis('off')

    plt.show()

    # 判别器准确率
    real_output = discriminator(test_data, training=False)
    fake_output = discriminator(generated_images, training=False)
    accuracy_real = np.mean(real_output > 0.5)
    accuracy_fake = np.mean(fake_output < 0.5)
    print("Discriminator Accuracy (Real):", accuracy_real)
    print("Discriminator Accuracy (Fake):", accuracy_fake)
    
#加载测试数据    
def load_cifar10_testdata():
    data = []
    labels = []
    filename = f'C:/Users/Starfish/Desktop/单佳震/cifar-10-batches-py/test_batch'
    with open(filename, 'rb') as f:
        batch = pickle.load(f, encoding='bytes')
        data.append(batch[b'data'])
        labels += batch[b'labels']
    data = np.concatenate(data)
    data = data.reshape((-1, img_channels, img_height, img_width)).transpose(0, 2, 3, 1) / 255.0
    labels = np.array(labels)
    return data, labels

# 使用测试数据集评估GAN模型
test_data, _ = load_cifar10_testdata()
evaluate_gan(generator, discriminator, test_data[:10])


这段代码的作用是评估一个已经训练好的GAN模型的生成能力和判别器的准确率。
1. evaluate_gan函数：该函数接收一个生成器模型、一个判别器模型和测试数据集作为输入，然后使用生成器模型生成10张图像并与对应的真实图像一起显示出来。接着，它通过判别器模型对这些图像进行分类，并计算出判别器在真实图像和伪造图像上的准确率。
2. load_cifar10_testdata函数：该函数从CIFAR-10测试数据集中加载测试图像和标签，并将数据归一化到[0, 1]范围内。

评估GAN模型的性能，了解在生成逼真图像方面的能力以及判别器准确率。通过这些信息，我们可以判断模型是否需要进行调整或改进。

## 对抗攻击实验

### 第一次对抗攻击

In [ ]:
def generate_adversarial_samples(data, generator, epsilon=0.1):
    adv_samples = []
    for image in data:
        noise = tf.random.normal([1, latent_dim])
        generated_image = generator(noise, training=False)

        perturbation = epsilon * (generated_image - image)
        adv_image = tf.clip_by_value(image + perturbation, -1, 1)
        adv_samples.append(adv_image)
    return tf.concat(adv_samples, axis=0)

# 生成对抗样本
epsilon = 0.5  # 调整扰动的大小
adversarial_samples = generate_adversarial_samples(test_data[:10], generator, epsilon)

# 可视化对抗样本和原始图像
plt.figure(figsize=(12, 4))
for i in range(10):
    plt.subplot(2, 10, i + 1)
    plt.imshow((test_data[i] + 1) / 2)
    plt.title('Original')
    plt.axis('off')
    
    plt.subplot(2, 10, i + 11)
    plt.imshow((adversarial_samples[i] + 1) / 2)
    plt.title('Adversarial')
    plt.axis('off')

plt.show()

def evaluate_model(model, data, labels):
    predictions = model.predict(data)
    accuracy = np.mean(np.argmax(predictions, axis=1) == labels)
    return accuracy

# 原始图像评估
original_accuracy = evaluate_model(discriminator, test_data[:10], np.ones(10))
print("Original Accuracy:", original_accuracy)


# 对抗样本评估
adversarial_accuracy = evaluate_model(discriminator, adversarial_samples, np.zeros(10))
print("Adversarial Accuracy:", adversarial_accuracy)


### 第二次对抗攻击

In [ ]:
# 生成对抗样本
epsilon = 0.3  # 调整扰动的大小
adversarial_samples = generate_adversarial_samples(test_data[10:20], generator, epsilon)

# 可视化对抗样本和原始图像
plt.figure(figsize=(12, 4))
for i in range(10):
    plt.subplot(2, 10, i + 1)
    plt.imshow((test_data[i+10] + 1) / 2)
    plt.title('Original')
    plt.axis('off')
    
    plt.subplot(2, 10, i + 11)
    plt.imshow((adversarial_samples[i] + 1) / 2)
    plt.title('Adversarial')
    plt.axis('off')

plt.show()

def evaluate_model(model, data, labels):
    predictions = model.predict(data)
    accuracy = np.mean(np.argmax(predictions, axis=1) == labels)
    return accuracy

# 原始图像评估
original_accuracy = evaluate_model(discriminator, test_data[10:20], np.ones(10))
print("Original Accuracy:", original_accuracy)


# 对抗样本评估
adversarial_accuracy = evaluate_model(discriminator, adversarial_samples, np.zeros(10))
print("Adversarial Accuracy:", adversarial_accuracy)

[![ee7c1db01caec59b8cfcf434e65980ba.png](https://i.postimg.cc/yd9FC6nh/ee7c1db01caec59b8cfcf434e65980ba.png)](https://postimg.cc/8JpFhS2s)